<a href="https://colab.research.google.com/github/Asif-Mustafa-svg/Cancer_Prediction_classifier/blob/main/sarimax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from statsmodels.tsa.stattools import adfuller
%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = 12, 10
import pandas as pd
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from itertools import product
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_ccf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.model_selection import train_test_split
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
df=pd.read_csv("annual_co2_including_land_use.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'annual_co2_including_land_use.csv'

In [ ]:
df.isnull().sum()

NameError: name 'df' is not defined

In [ ]:
df.describe()

In [ ]:
df['Year']= pd.to_datetime(df['Year'], format='%Y')
df.set_index('Year', inplace=True)

In [ ]:
df.head()

In [ ]:
df['CO2_emissions']=df['Annual CO₂ emissions including land-use change']

NameError: name 'df' is not defined

In [ ]:
top50 = df.groupby(['Entity'])['CO2_emissions'].mean().sort_values(ascending=False).index[:50]
height = df.groupby(['Entity'])['CO2_emissions'].mean().sort_values(ascending=False)[:50]

fig, ax = plt.subplots(figsize=(15,10))
plt.bar(x=top50 , height=height, color = 'mediumseagreen')

plt.title("Mean CO2 Emissions Per Year \n (1950-2022)", fontsize=20)
plt.ylabel('Mean Total CO2 Emission Per Year (Tonnes)', fontsize=15)
for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(14)
    tick.label.set_rotation('vertical')

# Threshold to indicate countries above 150 mean tonnes
threshold = 150
plt.axhline(y=threshold,linewidth=2, linestyle='dashed', color='mediumblue')

plt.show()

# Uncomment below code only to save image
#plt.savefig("images/mean_emissions_per_year_top50")

In [ ]:
top50 = df.groupby(['Entity'])['CO2_emissions'].mean().sort_values(ascending=True).index[:50]
height = df.groupby(['Entity'])['CO2_emissions'].mean().sort_values(ascending=True)[:50]

fig, ax = plt.subplots(figsize=(15,10))
plt.bar(x=top50 , height=height, color = 'mediumseagreen')

plt.title("Mean CO2 Emissions Per Year \n (1950-2022)", fontsize=20)
plt.ylabel('Mean Total CO2 Emission Per Year (Tonnes)', fontsize=15)
for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(14)
    tick.label.set_rotation('vertical')

# Threshold to indicate countries above 150 mean tonnes
threshold = 150
plt.axhline(y=threshold,linewidth=2, linestyle='dashed', color='mediumblue')

plt.show()

# Uncomment below code only to save image
#plt.savefig("images/mean_emissions_per_year_top50")

In [ ]:
df_bangladesh= df[df['Entity'] == "Bangladesh"]

In [ ]:
df_bangladesh.head()

In [ ]:
z_scores = (df_bangladesh['CO2_emissions'] - df_bangladesh['CO2_emissions'].mean()) / df_bangladesh['CO2_emissions'].std()

# Define threshold for outlier detection
threshold = 3  # Z-score greater than 3 or less than -3 is considered an outlier

# Detect outliers
outliers = np.abs(z_scores) > threshold

# Replace outliers with NaN
df_bangladesh.loc[outliers, 'CO2_emissions'] = np.nan

# Interpolate missing values
df_bangladesh['CO2_emissions'] = df_bangladesh['CO2_emissions'].interpolate(method='linear')

print(df_bangladesh)

In [ ]:
# Perform ADF test on 'Emissions' column
result = adfuller(df_bangladesh['CO2_emissions'])

# Extract and print test results
print('ADF Statistic:', result[0])
print('p-value:', result[1])
print('Critical Values:')
for key, value in result[4].items():
    print(f'   {key}: {value}')

In [ ]:
# Decompose the time series to understand trend and seasonality
from statsmodels.tsa.seasonal import seasonal_decompose

decomposition = seasonal_decompose(df_bangladesh['CO2_emissions'], model='additive', period=12)
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

plt.figure(figsize=(10, 8))
plt.subplot(411)
plt.plot(df_bangladesh.index, df_bangladesh['CO2_emissions'], label='Original')
plt.legend(loc='upper left')
plt.subplot(412)
plt.plot(df_bangladesh.index, trend, label='Trend')
plt.legend(loc='upper left')
plt.subplot(413)
plt.plot(df_bangladesh.index, seasonal, label='Seasonality')
plt.legend(loc='upper left')
plt.subplot(414)
plt.plot(df_bangladesh.index, residual, label='Residuals')
plt.legend(loc='upper left')
plt.tight_layout()

# Fit an ARIMA model
model = ARIMA(df_bangladesh['CO2_emissions'], order=(1,1,1))
results = model.fit()
print(results.summary())


In [ ]:
df_bangladesh['CO2_emissions_1']= df_bangladesh['CO2_emissions']-df_bangladesh['CO2_emissions'].shift(1)
result = adfuller(df_bangladesh['CO2_emissions'].dropna())

# Extract and print test results
print('ADF Statistic:', result[0])
print('p-value:', result[1])
print('Critical Values:')
for key, value in result[4].items():
    print(f'   {key}: {value}')

In [ ]:
df_bangladesh.head()
df_bangladesh.tail()

In [ ]:
df_bangladesh['CO2_emissions_1'].plot()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

decomposition = seasonal_decompose(df_bangladesh['CO2_emissions_1'].fillna(df_bangladesh['CO2_emissions_1'].mean()), model='additive',period=12)
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

plt.figure(figsize=(10, 8))
plt.subplot(411)
plt.plot(df_bangladesh.index, df_bangladesh['CO2_emissions_1'].fillna(df_bangladesh['CO2_emissions_1'].mean()), label='Original')
plt.legend(loc='upper left')
plt.subplot(412)
plt.plot(df_bangladesh.index, trend, label='Trend')
plt.legend(loc='upper left')
plt.subplot(413)
plt.plot(df_bangladesh.index, seasonal, label='Seasonality')
plt.legend(loc='upper left')
plt.subplot(414)
plt.plot(df_bangladesh.index, residual, label='Residuals')
plt.legend(loc='upper left')
plt.tight_layout()



In [ ]:
plt.figure(figsize=(12, 6))
plot_acf(df_bangladesh['CO2_emissions_1'].fillna(df_bangladesh['CO2_emissions_1'].mean()), lags=40, ax=plt.gca())
plt.title('Autocorrelation Function (ACF) - 1st Order Differenced Data')
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.show()

# Plot partial autocorrelation function for differenced data
plt.figure(figsize=(12, 6))
plot_pacf(df_bangladesh['CO2_emissions_1'].fillna(df_bangladesh['CO2_emissions_1'].mean()), lags=30, ax=plt.gca())
plt.title('Partial Autocorrelation Function (PACF) - 1st Order Differenced Data')
plt.xlabel('Lag')
plt.ylabel('Partial Autocorrelation')
plt.show()

In [ ]:
df_bangladesh.head()

In [ ]:
order = (2, 1, 0)  # Non-seasonal parameters
seasonal_order = (2, 1, 0, 12)  # Seasonal parameters


sarima_model = SARIMAX(df_bangladesh['CO2_emissions_1'], order=order, seasonal_order=seasonal_order)
sarima_results = sarima_model.fit()

# Model diagnostics
sarima_results.plot_diagnostics(figsize=(12, 8))
plt.show()

# Forecasting
forecast_steps = 12  # Number of steps ahead to forecast
forecast = sarima_results.forecast(steps=forecast_steps)

# Plotting
plt.plot(df_bangladesh.index,df_bangladesh['CO2_emissions_1'], label='Observed')
plt.plot(forecast.index, forecast, label='Forecast', color='red')
plt.xlabel('Date')
plt.ylabel('Emissions')
plt.legend()
plt.show()

In [ ]:
mae = mean_absolute_error(df_bangladesh['CO2_emissions_1'][-forecast_steps:], forecast)
print('Mean Absolute Error (MAE):', mae)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(df_bangladesh['CO2_emissions_1'][-forecast_steps:], forecast)
print('Mean Squared Error (MSE):', mse)

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)
print('Root Mean Squared Error (RMSE):', rmse)

In [ ]:
# Splitting the data into train and test sets
train_data, test_data = train_test_split(df_bangladesh['CO2_emissions_1'], test_size=0.2, shuffle=False)

# Define SARIMA model parameters
order = (2, 1, 0)         # Non-seasonal parameters
seasonal_order = (2, 1, 0, 12)  # Seasonal parameters

# Build and fit SARIMA model on training data
sarima_model = SARIMAX(train_data, order=order, seasonal_order=seasonal_order)
sarima_results = sarima_model.fit()

# Model diagnostics
sarima_results.plot_diagnostics(figsize=(12, 8))
plt.show()

# Forecasting
forecast_steps = len(test_data)  # Number of steps ahead to forecast is equal to the length of the test set
forecast = sarima_results.forecast(steps=forecast_steps)

# Plotting
plt.plot(df_bangladesh.index[:len(train_data)], train_data, label='Train data')
plt.plot(df_bangladesh.index[len(train_data):], test_data, label='Test data')
plt.plot(df_bangladesh.index[len(train_data):], forecast, label='Forecast', color='black')
plt.xlabel('Date')
plt.ylabel('Emissions')
plt.legend()
plt.show()

In [ ]:
# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(test_data, forecast)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(test_data, forecast)

print('Mean Absolute Error (MAE):', mae)
print('Mean Squared Error (MSE):', mse)

In [ ]:
df_bangladesh.head()

In [ ]:
df_bangladesh1=df_bangladesh['CO2_emissions']
df_bangladesh1.head()

In [ ]:
df_bangladesh.isnull().sum()
df_bangladesh.fillna(df_bangladesh['CO2_emissions_1'].mean())

In [ ]:

# Data preprocessing
scaler1 = StandardScaler()
scaled_data = scaler1.fit_transform(df_bangladesh['CO2_emissions'].values.reshape(-1, 1))

# Define function to create input sequences for LSTM
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:(i + seq_length), 0])
        y.append(data[i + seq_length, 0])
    return np.array(X), np.array(y)

# Set sequence length and split data into training and test sets
sequence_length = 12  # Adjust as needed
X, y = create_sequences(scaled_data, sequence_length)
X_train, X_test = X[:-12], X[-12:]
y_train, y_test = y[:-12], y[-12:]

# Reshape input data for LSTM
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Build LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(sequence_length, 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))

# Compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train model
model.fit(X_train, y_train, epochs=115, batch_size=32)

# Make predictions
predictions = model.predict(X_test)
predictions = scaler1.inverse_transform(predictions)

# Calculate accuracy scores
mae = mean_absolute_error(df_bangladesh['CO2_emissions'][-12:], predictions)
mse = mean_squared_error(df_bangladesh['CO2_emissions'][-12:], predictions)
rmse = np.sqrt(mse)

print('Mean Absolute Error (MAE):', mae)
print('Mean Squared Error (MSE):', mse)
print('Root Mean Squared Error (RMSE):', rmse)

# Plot results
plt.plot(df_bangladesh.index[-12:], df_bangladesh['CO2_emissions'][-12:], label='Actual')
plt.plot(df_bangladesh.index[-12:], predictions, label='Predicted', color='red')
plt.xlabel('Date')
plt.ylabel('Emissions')
plt.legend()
plt.show()

In [ ]:
scaler1 = StandardScaler()
scaled_data = scaler1.fit_transform(df_bangladesh['CO2_emissions'].values.reshape(-1, 1))

print("Length of input data:", len(scaled_data))

sequence_length = 50

# Adjust sequence length if it's larger than the length of input data
sequence_length = min(sequence_length, len(scaled_data))

# Create sequences of length 50
X, y = create_sequences(scaled_data, sequence_length)

# Check if there is sufficient data to create sequences
if len(X) == 0 or len(y) == 0:
    print("Error: Insufficient data to create sequences with the specified sequence length.")
else:
    # Split data into training and test sets
    X_train, X_test = X[:-12], X[-12:]
    y_train, y_test = y[:-12], y[-12:]

    # Reshape input data for LSTM
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

    # Build LSTM model
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(sequence_length, 1)))
    model.add(LSTM(units=50))
    model.add(Dense(units=1))

    # Compile model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train model
    model.fit(X_train, y_train, epochs=100, batch_size=32)

    # Make predictions
    predictions = model.predict(X_test)
    predictions = scaler1.inverse_transform(predictions)

    # Calculate accuracy scores
    mae = mean_absolute_error(df_bangladesh['CO2_emissions'][-12:], predictions)
    mse = mean_squared_error(df_bangladesh['CO2_emissions'][-12:], predictions)
    rmse = np.sqrt(mse)

    print('Mean Absolute Error (MAE):', mae)
    print('Mean Squared Error (MSE):', mse)
    print('Root Mean Squared Error (RMSE):', rmse)

    # Plot results
    plt.plot(df_bangladesh.index[-12:], df_bangladesh['CO2_emissions'][-12:], label='Actual')
    plt.plot(df_bangladesh.index[-12:], predictions, label='Predicted', color='red')
    plt.xlabel('Date')
    plt.ylabel('Emissions')
    plt.legend()
    plt.show()

In [ ]:
from keras.layers import Dropout

# Build LSTM model with adjusted architecture
model = Sequential()
model.add(LSTM(units=64, return_sequences=True, input_shape=(sequence_length, 1)))
model.add(Dropout(0.2))  # Add dropout regularization
model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(0.2))  # Add dropout regularization
model.add(LSTM(units=32))
model.add(Dropout(0.2))  # Add dropout regularization
model.add(Dense(units=1))

# Compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train model
model.fit(X_train, y_train, epochs=100, batch_size=32)

# Make predictions
predictions = model.predict(X_test)
predictions = scaler1.inverse_transform(predictions)

# Calculate accuracy scores
mae = mean_absolute_error(df_bangladesh['CO2_emissions'][-12:], predictions)
mse = mean_squared_error(df_bangladesh['CO2_emissions'][-12:], predictions)
rmse = np.sqrt(mse)

print('Mean Absolute Error (MAE):', mae)
print('Mean Squared Error (MSE):', mse)
print('Root Mean Squared Error (RMSE):', rmse)

# Plot results
plt.plot(df_bangladesh.index[-12:], df_bangladesh['CO2_emissions'][-12:], label='Actual')
plt.plot(df_bangladesh.index[-12:], predictions, label='Predicted', color='red')
plt.xlabel('Date')
plt.ylabel('Emissions')
plt.legend()
plt.show()

In [ ]:
df_electricity=pd.read_csv("electricity-prod-source-stacked.csv")

In [ ]:
df_electricity.head()

In [ ]:
df_electricity_bangladesh=df_electricity[df_electricity['Entity']=="Bangladesh"]

In [ ]:
df_electricity_bangladesh.describe()

In [ ]:
df_electricity_bangladesh.columns

In [ ]:
a = {
    'Other renewables excluding bioenergy - TWh (adapted for visualization of chart electricity-prod-source-stacked)':'renewables',
    'Electricity from bioenergy - TWh (adapted for visualization of chart electricity-prod-source-stacked)': "bioenergy",
    'Electricity from solar - TWh (adapted for visualization of chart electricity-prod-source-stacked)': 'solar',
    'Electricity from wind - TWh (adapted for visualization of chart electricity-prod-source-stacked)': 'wind',
    'Electricity from hydro - TWh (adapted for visualization of chart electricity-prod-source-stacked)': 'hydro',
    'Electricity from nuclear - TWh (adapted for visualization of chart electricity-prod-source-stacked)': 'nuclear',
    'Electricity from oil - TWh (adapted for visualization of chart electricity-prod-source-stacked)': 'oil',
    'Electricity from gas - TWh (adapted for visualization of chart electricity-prod-source-stacked)': 'gas',
    'Electricity from coal - TWh (adapted for visualization of chart electricity-prod-source-stacked)': 'coal'
}

# Rename the columns using the mapping dictionary
df_electricity_bangladesh = df_electricity_bangladesh.rename(columns=a)

In [ ]:
df_electricity_bangladesh.head()

In [ ]:
df1=df_electricity_bangladesh.copy()

df1['Year']= pd.to_datetime(df1['Year'], format='%Y')
df1.set_index('Year', inplace=True)

In [ ]:
df1.head()

In [ ]:
concatenated_df = pd.concat([df, df1], axis=0)

In [ ]:
concatenated_df.head()

In [ ]:
last_23_rows = concatenated_df.tail(23)

# Split the last 23 rows into a new DataFrame
new_df = pd.DataFrame(last_23_rows)

# Display the new DataFrame
print(new_df)

In [ ]:
new_df.head()

In [ ]:
new_df.to_excel("new_df.xlsx", index=False)

In [ ]:
last_23_rows = df_bangladesh.tail(23)
print(last_23_rows )

In [ ]:
new_df1 = pd.read_excel("new_df1.xlsx")

In [ ]:
new_df1.head()

In [ ]:
new_df1['Year']= pd.to_datetime(new_df1['Year'], format='%Y')
new_df1.set_index('Year', inplace=True)

In [ ]:
new_df1.head()

In [ ]:
# Split data into train and test sets
train_size = int(len(new_df1) * 0.8)
train, test = new_df1.iloc[:train_size],new_df1.iloc[train_size:]

# SARIMA model
order = (2, 1, 0)  # ARIMA order
seasonal_order = (2, 1, 0, 12)  # Seasonal order
sarima_model = SARIMAX(train['CO2_emissions'], order=order, seasonal_order=seasonal_order)
sarima_results = sarima_model.fit()
sarima_forecast = sarima_results.forecast(steps=len(test))
sarima_rmse = np.sqrt(mean_squared_error(test['CO2_emissions'], sarima_forecast))
print("SARIMA RMSE:", sarima_rmse)

# SARIMAX model with exogenous variables
exog_train = train[["renewables","bioenergy","solar","wind","hydro","nuclear","oil","gas","coal"]]  # Example: exogenous variables
exog_test = test[["renewables","bioenergy","solar","wind","hydro","nuclear","oil","gas","coal"]]
sarimax_model = SARIMAX(train['CO2_emissions'], order=order, seasonal_order=seasonal_order, exog=exog_train)
sarimax_results = sarimax_model.fit()
sarimax_forecast = sarimax_results.forecast(steps=len(test), exog=exog_test)
sarimax_rmse = np.sqrt(mean_squared_error(test['CO2_emissions'], sarimax_forecast))
print("SARIMAX RMSE:", sarimax_rmse)

# Machine learning-based model (Random Forest)
rf_model = RandomForestRegressor(n_estimators=100)
X_train, y_train = train[["renewables","bioenergy","solar","wind","hydro","nuclear","oil","gas","coal"]], train['CO2_emissions']
X_test, y_test = test[["renewables","bioenergy","solar","wind","hydro","nuclear","oil","gas","coal"]], test['CO2_emissions']
rf_model.fit(X_train, y_train)
rf_forecast = rf_model.predict(X_test)
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_forecast))
print("Random Forest RMSE:", rf_rmse)

In [ ]:
# Plot actual vs. predicted values for SARIMA
plt.figure(figsize=(10, 6))
plt.plot(test.index, test['CO2_emissions'], label='Actual')
plt.plot(test.index, sarima_forecast, label='SARIMA Forecast')
plt.title('Actual vs. SARIMA Forecast')
plt.xlabel('Date')
plt.ylabel('CO2 Emissions')
plt.legend()
plt.show()

# Plot actual vs. predicted values for SARIMAX
plt.figure(figsize=(10, 6))
plt.plot(test.index, test['CO2_emissions'], label='Actual')
plt.plot(test.index, sarimax_forecast, label='SARIMAX Forecast')
plt.title('Actual vs. SARIMAX Forecast')
plt.xlabel('Date')
plt.ylabel('CO2 Emissions')
plt.legend()
plt.show()

# Plot actual vs. predicted values for Random Forest
plt.figure(figsize=(10, 6))
plt.plot(test.index, test['CO2_emissions'], label='Actual')
plt.plot(test.index, rf_forecast, label='Random Forest Forecast')
plt.title('Actual vs. Random Forest Forecast')
plt.xlabel('Date')
plt.ylabel('CO2 Emissions')
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(new_df1['renewables'], new_df1['CO2_emissions'])
plt.title('Renewables vs. CO2 Emissions')
plt.xlabel('Renewables')
plt.ylabel('CO2 Emissions')
plt.grid(True)
plt.show()

# Scatter plot for bioenergy vs. CO2 emissions
plt.figure(figsize=(10, 6))
plt.scatter(new_df1['bioenergy'],new_df1['CO2_emissions'])
plt.title('Bioenergy vs. CO2 Emissions')
plt.xlabel('Bioenergy')
plt.ylabel('CO2 Emissions')
plt.grid(True)
plt.show()

# Repeat the above steps for solar, wind, hydro, nuclear, oil, gas, and coal columns
# ...

# Line plot showing the trend of each column over time
plt.figure(figsize=(10, 6))
for column in ['renewables', 'bioenergy', 'solar', 'wind', 'hydro', 'nuclear', 'oil', 'gas', 'coal']:
    plt.plot(new_df1.index, new_df1[column], label=column)
plt.title('Trend of Energy Sources Over Time')
plt.xlabel('Date')
plt.ylabel('Energy Production')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
new_df1.head(10)

In [ ]:

droppa=['Annual CO₂ emissions including land-use change','Entity','Code']
new_df1=new_df1.drop(droppa,axis=1)

In [ ]:
new_df1.head()

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(15, 10))

# Flatten the axs array to make it easier to access individual subplots
axs = axs.flatten()

# Plot the trend of each column with respect to CO2 emissions
for i, column in enumerate(['renewables', 'bioenergy', 'solar', 'wind', 'hydro', 'nuclear', 'oil', 'gas', 'coal']):
    axs[i].plot(new_df1.index, new_df1[column], label=column)
    axs[i].plot(new_df1.index, new_df1['CO2_emissions'], color='red', linestyle='--', label='CO2 Emissions')
    axs[i].set_title(f'Trend of {column.capitalize()} and CO2 Emissions Over Time')
    axs[i].set_xlabel('Date')
    axs[i].set_ylabel('Production / Emissions')
    axs[i].legend()
    axs[i].grid(True)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# Create subplots
fig, axs = plt.subplots(3, 3, figsize=(15, 10))

# Flatten the axs array to make it easier to access individual subplots
axs = axs.flatten()

# Plot the trend of each column with respect to CO2 emissions
for i, column in enumerate(['renewables', 'bioenergy', 'solar', 'wind', 'hydro', 'nuclear', 'oil', 'gas', 'coal']):
    axs[i].plot(new_df1.index, new_df1[column], label=column)
    axs[i].plot(new_df1.index, new_df1['CO2_emissions'], color='red', linestyle='--', label='CO2 Emissions')
    axs[i].set_title(f'Trend of {column.capitalize()} and CO2 Emissions Over Time')
    axs[i].set_xlabel('Date')
    axs[i].set_ylabel('Production / Emissions')
    axs[i].legend()
    axs[i].grid(True)

    # Set consistent y-axis limits for better visualization
    max_value = max(new_df1[column].max(), new_df1['CO2_emissions'].max())
    min_value = min(new_df1[column].min(), new_df1['CO2_emissions'].min())
    axs[i].set_ylim(min_value, max_value)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
for column in new_df1.columns:
    if column != 'CO2_emissions':
        plt.plot(new_df1['CO2_emissions'], new_df1[column], marker='o', linestyle='-', label=column)

# Add labels and title
plt.title('Trend of All Columns with Respect to CO2 Emissions Over Time')
plt.xlabel('CO2 Emissions')
plt.ylabel('Value')
plt.legend()
plt.grid(True)

# Show plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Assuming df is your DataFrame

# Create subplots
fig, axes = plt.subplots(2, 1, figsize=(10, 10))

# Plot CO2 emissions
axes[0].plot(new_df1.index,new_df1['CO2_emissions'], color='blue', marker='o', linestyle='-', label='CO2 Emissions')
axes[0].set_title('CO2 Emissions Over Time')
axes[0].set_xlabel('Year')
axes[0].set_ylabel('CO2 Emissions')
axes[0].legend()
axes[0].grid(True)

# Plot other columns
for column in new_df1.columns:
    if column != 'CO2_emissions':
        axes[1].plot(new_df1.index, new_df1[column],new_df1['CO2_emissions'], label=column)

axes[1].set_title('Trend of Other Columns Over Time')
axes[1].set_xlabel('Year')
axes[1].set_ylabel('Value')
axes[1].legend()
axes[1].grid(True)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
new_df1.isnull().sum()

In [ ]:
columns_to_scale = ['renewables', 'bioenergy', 'solar', 'wind', 'hydro', 'nuclear', 'oil', 'gas', 'coal', 'CO2_emissions']

# Create a StandardScaler object
scaler = StandardScaler()

# Fit the scaler to the selected columns and transform the data
scaled_data = scaler.fit_transform(new_df1[columns_to_scale])

# Convert the scaled data back to a DataFrame
scaled_new_df = pd.DataFrame(scaled_data, columns=columns_to_scale, index=new_df1.index)


In [ ]:
fig, axes = plt.subplots(len(new_df1.columns) - 1, 1, figsize=(10, 10))

# Plot each column with respect to CO2 emissions
for i, column in enumerate(new_df1.columns):
    if column != 'CO2_emissions':
        axes[i - 1].plot(new_df1.index, new_df1[column], label=column)
        axes[i - 1].plot(new_df1.index, new_df1['CO2_emissions'], color='red', linestyle='--', label='CO2 Emissions')
        axes[i - 1].set_title(f'Trend of {column.capitalize()} and CO2 Emissions Over Time')
        axes[i - 1].set_xlabel('Year')
        axes[i - 1].set_ylabel('Value')
        axes[i - 1].legend()
        axes[i - 1].grid(True)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(10, 10))

# Plot trend of solar production
axs[0].plot(new_df1.index, new_df1['solar'], color='orange', marker='o', linestyle='-', label='Solar Production')
axs[0].set_title('Trend of Solar Production Over Time')
axs[0].set_xlabel('Year')
axs[0].set_ylabel('Solar Production')
axs[0].legend()
axs[0].grid(True)

# Plot trend of CO2 emissions
axs[1].plot(new_df1.index, new_df1['CO2_emissions'], color='blue', marker='o', linestyle='-', label='CO2 Emissions')
axs[1].set_title('Trend of CO2 Emissions Over Time')
axs[1].set_xlabel('Year')
axs[1].set_ylabel('CO2 Emissions')
axs[1].legend()
axs[1].grid(True)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))

# Plot trend of solar production
plt.plot(scaled_new_df.index, scaled_new_df['solar'], color='orange', marker='o', linestyle='-', label='Solar Production')

# Plot trend of CO2 emissions
plt.plot(scaled_new_df.index, scaled_new_df['CO2_emissions'], color='blue', marker='o', linestyle='-', label='CO2 Emissions')

# Add titles and labels
plt.title('Trend of Solar Production and CO2 Emissions Over Time')
plt.xlabel('Year')
plt.ylabel('Value')

# Add a legend
plt.legend()

# Add gridlines
plt.grid(True)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Create subplots
fig, axs = plt.subplots(3, 3, figsize=(15, 10))

# Flatten the axs array to make it easier to access individual subplots
axs = axs.flatten()

# Plot the trend of each column with respect to CO2 emissions
for i, column in enumerate(['renewables', 'bioenergy', 'solar', 'wind', 'hydro', 'nuclear', 'oil', 'gas', 'coal']):
    axs[i].plot(scaled_new_df.index, scaled_new_df[column], label=column)
    axs[i].plot(scaled_new_df.index, scaled_new_df['CO2_emissions'], color='red', linestyle='--', label='CO2 Emissions')
    axs[i].set_title(f'Trend of {column.capitalize()} and CO2 Emissions Over Time')
    axs[i].set_xlabel('Date')
    axs[i].set_ylabel('Production / Emissions')
    axs[i].legend()
    axs[i].grid(True)



# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# Split data into train and test sets
train_size = int(len(scaled_new_df) * 0.8)
train, test = scaled_new_df.iloc[:train_size],scaled_new_df.iloc[train_size:]

# SARIMA model
order = (2, 1, 0)  # ARIMA order
seasonal_order = (2, 1, 0, 12)  # Seasonal order
sarima_model = SARIMAX(train['CO2_emissions'], order=order, seasonal_order=seasonal_order)
sarima_results = sarima_model.fit()
sarima_forecast = sarima_results.forecast(steps=len(test))
sarima_rmse = np.sqrt(mean_squared_error(test['CO2_emissions'], sarima_forecast))
print("SARIMA RMSE:", sarima_rmse)

# SARIMAX model with exogenous variables
exog_train = train[["renewables","bioenergy","solar","wind","hydro","nuclear","oil","gas","coal"]]  # Example: exogenous variables
exog_test = test[["renewables","bioenergy","solar","wind","hydro","nuclear","oil","gas","coal"]]
sarimax_model = SARIMAX(train['CO2_emissions'], order=order, seasonal_order=seasonal_order, exog=exog_train)
sarimax_results = sarimax_model.fit()
sarimax_forecast = sarimax_results.forecast(steps=len(test), exog=exog_test)
sarimax_rmse = np.sqrt(mean_squared_error(test['CO2_emissions'], sarimax_forecast))
print("SARIMAX RMSE:", sarimax_rmse)

# Machine learning-based model (Random Forest)
rf_model = RandomForestRegressor(n_estimators=100)
X_train, y_train = train[["renewables","bioenergy","solar","wind","hydro","nuclear","oil","gas","coal"]], train['CO2_emissions']
X_test, y_test = test[["renewables","bioenergy","solar","wind","hydro","nuclear","oil","gas","coal"]], test['CO2_emissions']
rf_model.fit(X_train, y_train)
rf_forecast = rf_model.predict(X_test)
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_forecast))
print("Random Forest RMSE:", rf_rmse)

In [ ]:
plt.plot(test.index, test['CO2_emissions'], label='Actual', color='black')

# Plotting SARIMA forecast
plt.plot(test.index, sarima_forecast, label='SARIMA Forecast', color='blue')

# Plotting SARIMAX forecast
plt.plot(test.index, sarimax_forecast, label='SARIMAX Forecast', color='green')

# Plotting Random Forest forecast
plt.plot(test.index, rf_forecast, label='Random Forest Forecast', color='red')

plt.xlabel('Date')
plt.ylabel('CO2 Emissions')
plt.title('Forecasting CO2 Emissions')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(scaled_new_df['renewables'], scaled_new_df['CO2_emissions'])
plt.title('Renewables vs. CO2 Emissions')
plt.xlabel('Renewables')
plt.ylabel('CO2 Emissions')
plt.grid(True)
plt.show()

# Scatter plot for bioenergy vs. CO2 emissions
plt.figure(figsize=(10, 6))
plt.scatter(scaled_new_df['bioenergy'],scaled_new_df['CO2_emissions'])
plt.title('Bioenergy vs. CO2 Emissions')
plt.xlabel('Bioenergy')
plt.ylabel('CO2 Emissions')
plt.grid(True)
plt.show()

# Repeat the above steps for solar, wind, hydro, nuclear, oil, gas, and coal columns
# ...

# Line plot showing the trend of each column over time
plt.figure(figsize=(10, 6))
for column in ['renewables', 'bioenergy', 'solar', 'wind', 'hydro', 'nuclear', 'oil', 'gas', 'coal']:
    plt.plot(scaled_new_df.index, scaled_new_df[column], label=column)
plt.title('Trend of Energy Sources Over Time')
plt.xlabel('Date')
plt.ylabel('Energy Production')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Create exogenous variables for the next 10 years
future_exog = pd.DataFrame(index=pd.date_range(start=test.index[-1], periods=10, freq='Y'), columns=exog_train.columns)

# SARIMAX forecast for the next 10 years
sarimax_future_forecast = sarimax_results.forecast(steps=10, exog=future_exog)

# Random Forest forecast for the next 10 years
rf_future_forecast = rf_model.predict(future_exog)

# Print the forecasted values
print("\nSARIMAX Future Forecast:\n", sarimax_future_forecast)
print("\nRandom Forest Future Forecast:\n", rf_future_forecast)